## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [5]:

#from google.colab import drive
#drive.mount('/gdrive')

In [6]:
import pandas as pd

In [7]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [9]:
data.drop(columns=["date","symbol"],inplace=True)

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [12]:
data_1000=data.head(n=1000)

In [13]:
data_1000.shape

(1000, 5)

In [ ]:
data_1000["volume"]=data_1000["volume"]/1000

In [23]:
data_1000.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163.6
1,125.239998,119.980003,119.940002,125.540001,2386.4
2,116.379997,114.949997,114.930000,119.739998,2489.5
3,115.480003,116.620003,113.500000,117.440002,2006.3
4,117.010002,114.970001,114.089996,117.330002,1408.6


### Divide the data into train and test sets

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X=data_1000.drop(columns="volume")
Y=data_1000["volume"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [29]:
import numpy as np

In [45]:
#X_train_arr=np.asarray(X_train)
#X_test_arr=np.asarray(X_test)
X_train_arr=np.float32(X_train)
X_test_arr=np.float32(X_test)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [46]:
from sklearn.preprocessing import scale

In [47]:
X_train_std=scale(X_train_arr)
X_test_std=scale(X_test_arr)

C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [48]:
X_train_std

array([[-0.6471594 , -0.65192837, -0.6477564 , -0.6535503 ],
       [-0.8951312 , -0.89168936, -0.89293355, -0.89432305],
       [-0.8009922 , -0.7985978 , -0.79978985, -0.80114216],
       ...,
       [ 9.375546  ,  9.332643  ,  9.377511  ,  9.330112  ],
       [ 0.9817958 ,  0.94452477,  0.94749826,  0.9782794 ],
       [ 0.77863616,  0.7824517 ,  0.7736976 ,  0.8027333 ]],
      dtype=float32)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [58]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
#x_n = tf.nn.l2_normalize(x,1)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

In [68]:
W = tf.Variable(tf.random_normal(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

In [108]:
W

<tf.Variable 'Weights_4:0' shape=(4, 1) dtype=float32_ref>

2.Define a function to calculate prediction

In [69]:
y = tf.add(tf.matmul(x,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [70]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')


4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [94]:
train_op = tf.train.GradientDescentOptimizer(0.3).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [109]:
W_arr=[]
b_arr=[]
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [110]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train_std, y_:y_train})
    W_arr.append(sess.run(W))
    b_arr.append(sess.run(b))
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  225879650.0
Training loss at step:  5  is  195406770.0
Training loss at step:  10  is  195403550.0
Training loss at step:  15  is  195403680.0
Training loss at step:  20  is  195408100.0
Training loss at step:  25  is  195536290.0
Training loss at step:  30  is  199240800.0
Training loss at step:  35  is  306292860.0
Training loss at step:  40  is  3399875800.0
Training loss at step:  45  is  92797950000.0
Training loss at step:  50  is  2676219400000.0
Training loss at step:  55  is  77331420000000.0
Training loss at step:  60  is  2234719100000000.0
Training loss at step:  65  is  6.4578858e+16
Training loss at step:  70  is  1.8661563e+18
Training loss at step:  75  is  5.392762e+19
Training loss at step:  80  is  1.5583935e+21
Training loss at step:  85  is  4.5034024e+22
Training loss at step:  90  is  1.3014132e+24
Training loss at step:  95  is  3.7608463e+25


In [118]:
# Storing necessary values to be used outside the Session 
#training_cost = sess.run([train_op,loss], feed_dict ={X: X_train_std, Y: y_train}) 
weight = sess.run(W) 
bias = sess.run(b) 

### Get the shapes and values of W and b

In [111]:
#sess.run(W)
W_arr

[array([[-0.407263  ],
        [-1.0919313 ],
        [ 0.73944336],
        [ 0.6458762 ]], dtype=float32), array([[-0.33905852],
        [-1.0236696 ],
        [ 0.80767876],
        [ 0.71411276]], dtype=float32), array([[-0.43451083],
        [-1.1190796 ],
        [ 0.71223617],
        [ 0.6186856 ]], dtype=float32), array([[-0.30092365],
        [-0.98544866],
        [ 0.8458213 ],
        [ 0.75228995]], dtype=float32), array([[-0.48797256],
        [-1.1724561 ],
        [ 0.6587634 ],
        [ 0.56525093]], dtype=float32), array([[-0.22615722],
        [-0.9105984 ],
        [ 0.9205704 ],
        [ 0.8270819 ]], dtype=float32), array([[-0.5926807 ],
        [-1.2770809 ],
        [ 0.55403614],
        [ 0.4605662 ]], dtype=float32), array([[-0.07963157],
        [-0.76398885],
        [ 1.0670769 ],
        [ 0.97363305]], dtype=float32), array([[-0.7978616 ],
        [-1.4821789 ],
        [ 0.34883487],
        [ 0.25540668]], dtype=float32), array([[ 0.20753837],
     

In [113]:
b_arr

[array([3312.3079], dtype=float32),
 array([4637.2305], dtype=float32),
 array([5167.1997], dtype=float32),
 array([5379.1875], dtype=float32),
 array([5463.9824], dtype=float32),
 array([5497.9004], dtype=float32),
 array([5511.468], dtype=float32),
 array([5516.8945], dtype=float32),
 array([5519.0654], dtype=float32),
 array([5519.9336], dtype=float32),
 array([5520.2812], dtype=float32),
 array([5520.42], dtype=float32),
 array([5520.4756], dtype=float32),
 array([5520.498], dtype=float32),
 array([5520.507], dtype=float32),
 array([5520.5103], dtype=float32),
 array([5520.5117], dtype=float32),
 array([5520.512], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32),
 array([5520.5127], dtype=float32)

In [156]:
sess.close()

In [157]:
# Calculating the predictions 
predictions = np.matmul(X_test_std[1],weight) + bias 
print("Training cost =", train_loss, "Weight =", weight, "bias =", bias, '\n') 

Training cost = 5.54597e+26 Weight = [[8.241886e+12]
 [8.241905e+12]
 [8.241903e+12]
 [8.241984e+12]] bias = [3584948.5] 



In [166]:
tf.__version__

'1.13.1'

### Model Prediction on 1st Examples in Test Dataset

In [158]:
predictions


array([-6.524173e+12], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

In [193]:
df_iris=pd.read_csv("Iris (2).csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [194]:
df_iris["Species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [195]:
from sklearn.preprocessing import LabelEncoder

In [196]:
lb=LabelEncoder()

In [197]:
df_iris['Species']= lb.fit_transform(df_iris['Species'])

In [198]:
df_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [199]:
df_iris_dum=pd.get_dummies(df_iris,columns=["Species"])

### Splitting the data into feature set and target set

In [205]:
X=df_iris_dum.drop(columns=["Id","Species_0","Species_1","Species_2"])

In [206]:
Y=df_iris_dum.iloc[:,5:8]

In [207]:
Y.head()

,Species_0,Species_1,Species_2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### Divide the dataset into Training and test (70:30)

In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [209]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [226]:
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Model Training 

In [227]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=1)

Train on 105 samples, validate on 45 samples
Epoch 1/150
105/105 [==============================] - 0s 4ms/step - loss: 0.8175 - acc: 0.5714 - val_loss: 0.5444 - val_acc: 0.6000
Epoch 2/150
105/105 [==============================] - 0s 1ms/step - loss: 0.5185 - acc: 0.7524 - val_loss: 0.4924 - val_acc: 0.6222
Epoch 3/150
105/105 [==============================] - 0s 1ms/step - loss: 0.4407 - acc: 0.7619 - val_loss: 0.4042 - val_acc: 0.9333
Epoch 4/150
105/105 [==============================] - 0s 1ms/step - loss: 0.4118 - acc: 0.8000 - val_loss: 0.3976 - val_acc: 0.7556
Epoch 5/150
105/105 [==============================] - 0s 1ms/step - loss: 0.3641 - acc: 0.8762 - val_loss: 0.4086 - val_acc: 0.7111
Epoch 6/150
105/105 [==============================] - 0s 1ms/step - loss: 0.3453 - acc: 0.8571 - val_loss: 0.3345 - val_acc: 0.8444
Epoch 7/150
105/105 [==============================] - 0s 1ms/step - loss: 0.3099 - acc: 0.9143 - val_loss: 0.2743 - val_acc: 0.9556
Epoch 8/150
105/105 [===

105/105 [==============================] - 0s 951us/step - loss: 0.1344 - acc: 0.9619 - val_loss: 0.1460 - val_acc: 0.9333
Epoch 62/150
105/105 [==============================] - 0s 913us/step - loss: 0.1298 - acc: 0.9429 - val_loss: 0.4074 - val_acc: 0.8000
Epoch 63/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1271 - acc: 0.9333 - val_loss: 0.0698 - val_acc: 1.0000
Epoch 64/150
105/105 [==============================] - 0s 913us/step - loss: 0.1195 - acc: 0.9524 - val_loss: 0.1656 - val_acc: 0.9111
Epoch 65/150
105/105 [==============================] - 0s 951us/step - loss: 0.1192 - acc: 0.9619 - val_loss: 0.1457 - val_acc: 0.9333
Epoch 66/150
105/105 [==============================] - 0s 878us/step - loss: 0.1472 - acc: 0.9524 - val_loss: 0.0691 - val_acc: 1.0000
Epoch 67/150
105/105 [==============================] - 0s 920us/step - loss: 0.1188 - acc: 0.9524 - val_loss: 0.0712 - val_acc: 0.9778
Epoch 68/150
105/105 [==============================] - 0s 957u

105/105 [==============================] - 0s 989us/step - loss: 0.1174 - acc: 0.9619 - val_loss: 0.0611 - val_acc: 0.9778
Epoch 122/150
105/105 [==============================] - 0s 840us/step - loss: 0.1268 - acc: 0.9524 - val_loss: 0.1044 - val_acc: 0.9556
Epoch 123/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1108 - acc: 0.9619 - val_loss: 0.1354 - val_acc: 0.9333
Epoch 124/150
105/105 [==============================] - 0s 951us/step - loss: 0.1135 - acc: 0.9619 - val_loss: 0.1577 - val_acc: 0.9333
Epoch 125/150
105/105 [==============================] - 0s 951us/step - loss: 0.1332 - acc: 0.9524 - val_loss: 0.0587 - val_acc: 1.0000
Epoch 126/150
105/105 [==============================] - 0s 908us/step - loss: 0.1006 - acc: 0.9524 - val_loss: 0.1464 - val_acc: 0.9333
Epoch 127/150
105/105 [==============================] - 0s 909us/step - loss: 0.1089 - acc: 0.9429 - val_loss: 0.0787 - val_acc: 0.9778
Epoch 128/150
105/105 [==============================] - 

### Model Prediction

In [230]:
y_predict=model.predict(X_test)

In [235]:
from sklearn.metrics import confusion_matrix

In [239]:
confusion_matrix(Y_test,y_predict)

NameError: name 'Y_test' is not defined

### Save the Model

In [229]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [224]:
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [225]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=1)

Train on 105 samples, validate on 45 samples
Epoch 1/150
105/105 [==============================] - 0s 5ms/step - loss: 0.7296 - acc: 0.6857 - val_loss: 0.5469 - val_acc: 0.6000
Epoch 2/150
105/105 [==============================] - 0s 1ms/step - loss: 0.4632 - acc: 0.7524 - val_loss: 0.4158 - val_acc: 0.9556
Epoch 3/150
105/105 [==============================] - 0s 994us/step - loss: 0.4018 - acc: 0.8095 - val_loss: 0.3491 - val_acc: 0.8000
Epoch 4/150
105/105 [==============================] - 0s 1ms/step - loss: 0.3451 - acc: 0.8381 - val_loss: 0.3149 - val_acc: 0.8889
Epoch 5/150
105/105 [==============================] - 0s 1ms/step - loss: 0.3291 - acc: 0.8571 - val_loss: 0.2117 - val_acc: 1.0000
Epoch 6/150
105/105 [==============================] - 0s 1ms/step - loss: 0.3049 - acc: 0.8381 - val_loss: 0.4017 - val_acc: 0.7556
Epoch 7/150
105/105 [==============================] - 0s 1ms/step - loss: 0.3179 - acc: 0.8667 - val_loss: 0.3946 - val_acc: 0.7556
Epoch 8/150
105/105 [=

Epoch 62/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1447 - acc: 0.9524 - val_loss: 0.0594 - val_acc: 1.0000
Epoch 63/150
105/105 [==============================] - 0s 915us/step - loss: 0.1144 - acc: 0.9714 - val_loss: 0.1322 - val_acc: 0.9333
Epoch 64/150
105/105 [==============================] - 0s 1ms/step - loss: 0.0618 - acc: 0.9619 - val_loss: 0.0452 - val_acc: 1.0000
Epoch 65/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1295 - acc: 0.9619 - val_loss: 0.0649 - val_acc: 0.9778
Epoch 66/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1311 - acc: 0.9333 - val_loss: 0.0559 - val_acc: 1.0000
Epoch 67/150
105/105 [==============================] - 0s 1ms/step - loss: 0.0982 - acc: 0.9524 - val_loss: 0.0751 - val_acc: 0.9778
Epoch 68/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1465 - acc: 0.9619 - val_loss: 0.1062 - val_acc: 0.9333
Epoch 69/150
105/105 [==============================] - 0s 9

Epoch 123/150
105/105 [==============================] - 0s 990us/step - loss: 0.1219 - acc: 0.9714 - val_loss: 0.0700 - val_acc: 0.9778
Epoch 124/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1508 - acc: 0.9429 - val_loss: 0.1112 - val_acc: 0.9333
Epoch 125/150
105/105 [==============================] - 0s 877us/step - loss: 0.1239 - acc: 0.9619 - val_loss: 0.0967 - val_acc: 0.9333
Epoch 126/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1552 - acc: 0.9238 - val_loss: 0.0721 - val_acc: 1.0000
Epoch 127/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1126 - acc: 0.9619 - val_loss: 0.0731 - val_acc: 0.9778
Epoch 128/150
105/105 [==============================] - 0s 1ms/step - loss: 0.1155 - acc: 0.9429 - val_loss: 0.1175 - val_acc: 0.9333
Epoch 129/150
105/105 [==============================] - 0s 953us/step - loss: 0.1133 - acc: 0.9524 - val_loss: 0.0540 - val_acc: 0.9778
Epoch 130/150
105/105 [==========================